### Ejercicio 2

Desarrollar un notebook de Jupyter, denominado “hashtags.ipynb”, en el que se utilice como fuente de datos Kafka, y en concreto el topic kafkaTwitter. 

La duración del batch será de 5 segundos. 

Se procesarán los tweets que lleguen para extraer los hashtags que contengan (tener en cuenta que todos los hashtags comienzan por el carácter ‘#’).

Se irán mostrando, cada vez que se procese el batch (5 segundos) los diez hashtags más utilizados desde el inicio del programa hasta ese momento y el número total de apariciones de cada uno, ordenados de mayor a menor frecuencia.

In [161]:
import findspark
import requests
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
from pyspark.streaming.kafka import KafkaUtils
from collections import namedtuple
import pandas as pd

In [162]:
# inicializo spark
findspark.init('/home/j/spark-2.4.7-bin-hadoop2.7')

In [164]:
# necesario para spark streaming y kafka
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.7 pyspark-shell'

In [166]:
# creación del nombre del topic
topic = 'kafkaTwitter'

In [167]:
# inicialización de los contextos de Spark
sc = SparkContext("local[*]")
ssc = StreamingContext(sc, 5)
sqlContext = SQLContext(sc)

In [168]:
# inicialización del Dstream de kafka
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {
                        'bootstrap.servers':'localhost:9092, localhost:9093',
                        'group.id':'test-group'})

In [169]:
# ventana de recepción de tweets de 20 segundos
lines = kafkaStream.window( 20 )

In [170]:
# estructura de la tupla tweets: tags / count
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )

In [171]:
# Obtención de los tweets
( lines.flatMap( lambda text: text[1].split( " " ) ) # Dividimos por espacio
  .filter( lambda word: word.lower().startswith("#") ) # Obtenemos los tags
  .map( lambda word: ( word.lower(), 1 ) ) # transformamos a minúsculas la palabra
  .reduceByKey( lambda a, b: a + b ) # reducimos
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # almacenamos en un objeto Tweet
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Los ordenamos en un DataFrame
  .limit(10).registerTempTable("tweets") ) ) # registrado en una tabla

In [173]:
# inicializamos stream
ssc.start()

In [ ]:
# creación de dataframe que contendrá el top
top_total = pd.DataFrame()

# ejecución mientras sea True
while True:
    # ponemos un sleep entre ejecuciones
    time.sleep( 3 )
    
    # guardamos el top de tweets con tag y count
    top_10_tweets = sqlContext.sql( 'Select tag, count from tweets' )
    
    # pasamos a dataframe el top
    top_10_df = top_10_tweets.toPandas()
    
    # acumulamos a un top total para tener el resultado esperado del ejercicio
    top_total = top_total.append(top_10_df)
    
    # imprimimos
    print(top_total.groupby(['tag']).sum().sort_values('count', ascending=False))
    
    # borramos entre ejecuciones para ir viendo el resultado acumulado nada mas
    display.clear_output(wait=True)
    
    # nota: dejo un output para que se vea la ejecución

                    count
tag                      
#forocambio11j         26
#ciencia               15
#internet              14
#arte                   8
#uam                    8
#summerslam             8
#tuitutil               6
#orgullo2015            5
#cmin                   4
#estudiantes            4
#mtvhottest             2
#empleo                 2
#errejónenrne           2
#guipúzcoa              1
#xespaña”               1
#xelfuturodeespaña      1
#competencias,          1
#destacada              1
#tipografía             1
#tablondeanuncios       1
#rai                    1
#minecraft              1
#propuestas             1
#pc                     1
#osx                    1
#noestoyllorando        1
#fans                   1
#msg4321...en           1
#zumba                  1


In [160]:
sc.stop()
ssc.stop()